<a href="https://colab.research.google.com/github/RomainPtre/Jedha-projet/blob/main/TD/TD_12_07_01_Linear_regression_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doing a Linear regression without Scikit Learn

There's nothing better to understand the gradient descent algorithm than to code it from scratch. It may seem difficult at first glance but don't worry, we will guide you step by step. It is also a fantastic occasion to practice python key concepts like **Classes**!

Don't hesitate to come back to your Machine Learning course on linear regression to refresh your memory.

Our goal will be to code a simple linear regression such as :

$f(x) = \beta_1 \times x + \beta_0$

* Import the following libraries:
  * Numpy

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

* Define a `Model` class that will take two methods:
  1. `__init__(self)`, the class builder which will allow you to define an attribute $\beta_0$ (`beta_0` in your code) and an attribute $\beta_1$ (`beta_1` in your code). These attributes represent the coefficients/parameters of the model an we will be initialize them randomly using Numpy (cf: `np.random.randn`).
  2. `__call__(self, x)`, a special method that will turn our class into a callable which will return $\beta_1 \times x + \beta_0$ when called.

In [2]:
class Model():
  def __init__(self):   # le constructeur de la classe
    self.beta_0 = np.random.randn(1)  # on donne 1 comme argument pour avoir des array en sortie, si on met 0 on aura des float, ca peut poser probleme avec les ope rations qui viendront plus tard
    self.beta_1 = np.random.randn(1)

  def __call__(self, x):
    return self.beta_1* x + self.beta_0

* Create an instance of your class `Model`

In [3]:
model = Model()

In [4]:
model.__call__(2)   # j'ai prix un x au hasard là, pas besoin d'e crire __call__ ici car model est un callable maintenant, on peut juste faire model(2)


array([0.56245572])

* Try doing a first "regression" by running `model(3.0)`.
NB: If you don't have the same values as this notebook in output, this is normal since you have initialized your values randomly.

In [5]:
model(3.0)

array([0.55081127])

In [6]:
model.beta_1 * 3 + model.beta_0 # idem !

array([0.55081127])

* This value corresponds to a random prediction of your model. But we don't have any data yet. This time, let's use `sklearn` to import data.
  * Import `sklearn.datasets`
  * Use the `load_diabetes()` function to load the diebetes dataset in an object called `diabetes`.
  * Print the `DESCR` attribute of the diabetes object
  * Save the content of the `data` attribute in an object named `diabetes_data`
  * Save the content of the `target` attribute in an object named `y`

In [7]:
import sklearn.datasets as sk

In [8]:
diabetes = sk.load_diabetes()

In [9]:
type(diabetes)

sklearn.utils._bunch.Bunch

In [10]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [11]:
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])

In [12]:
diabetes_data = diabetes.data
y = diabetes.target

* We have too much data in this dataset `diabetes_data`, take only the third column of the dataset and store it in a `diabetes_X` variable.

In [13]:
diabetes_data.shape

(442, 10)

In [14]:
diabetes_X = diabetes_data[:,2]

In [15]:
diabetes_X[:10]

array([ 0.06169621, -0.05147406,  0.04445121, -0.01159501, -0.03638469,
       -0.04069594, -0.04716281, -0.00189471,  0.06169621,  0.03906215])

* Visualize your data using `plotly`.

In [16]:
diabetes_Xlab = diabetes.feature_names[2] # because we extracted only the third column of data
diabetes_Xlab

'bmi'

In [17]:
px.scatter(x=diabetes_X, y=y, labels={'x': 'BMI','y' : 'target'})

* Now we need to define a cost function. For a linear regression, we could use MSE :

`np.mean((model(input) - y)**2)`

  * Create this function which we'll call `mse` (for mean square error). This function will take two arguments `y_pred` & `y_true`.

In [18]:
def mse(y_pred, y_true):
  return np.mean((y_pred- y_true)**2)

* Test your function by inserting `model(diabetes_X)` & `y` as arguments.
* Calculate the rmse as well

In [19]:
mse(model(diabetes_X), y)

28896.652283035422

* Visualize your regression in relation to your points

In [20]:
reg_line = px.line(x=diabetes_X, y=model(diabetes_X))

plot = px.scatter(x=diabetes_X, y=y, labels={'x': 'BMI','y' : 'target'})
reg_line.update_traces(line=dict(color='red'))
plot.add_trace(reg_line.data[0])
plot.show()

* We're going to need to compute the gradients for our variable `model.beta_1` and our constant `model.beta_0`. To do this, we're going to need to review our derivative formulas. Since we're not here to do math, we're going to give you these formulas.
  * `derive_model_beta_1 = 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))`
  * `derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))`

  * Feel free to read this article if you want to know more about the calculation of the derivative: [Gradient Descent Derivation](https://mccormickml.com/2014/03/04/gradient-descent-derivation/)


  * So using the above formulas, code the first function `derivative_mse_beta_1` that will take the arguments:
    * `x` --> the values for your variable / `y_pred` --> the values predicted by your model / `y_true` --> the values of the target variable


In [21]:
def derivative_mse_beta_1(x, y_pred, y_true):
  return 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))  # @ is a matrix multiplication


* Test you function

In [22]:
derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

-4.296139840873984

* So using the above formulas, now code the `derivative_mse_beta_0` function which will take the arguments :
    * `y_pred` --> the values predicted by your model / `y_true` --> the actual values to predict

In [23]:
def derivative_mse_beta_0(y_pred, y_true):
  return 2/len(y_pred)*(np.sum(y_pred - y_true))

* Test you function

In [24]:
derivative_mse_beta_0(model(diabetes_X), y)

-303.09547908373685

* We will try to see if we can minimize our cost function using the two gradients above. To update our variables, we need to subtract their respective gradients. Ex:
  * `param = param - learning_rate * gradient`

  * Set a `learning_rate` to 0.1
  * Try to apply your formula on `model.beta_1` and `model.beta_0`.

In [25]:
learning_rate = 0.1

In [26]:
print('old model beta 1 : {}'.format(model.beta_1))
print('old model beta 0: {}'.format(model.beta_0))

old model beta 1 : [-0.01164445]
old model beta 0: [0.58574462]


In [27]:
model.beta_1 -= learning_rate * derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)
model.beta_0 -= learning_rate * derivative_mse_beta_0(model(diabetes_X), y)

In [28]:
print('new model beta 1 : {}'.format(model.beta_1))
print('new model beta 0: {}'.format(model.beta_0))

new model beta 1 : [0.41796953]
new model beta 0: [30.89529253]


In [29]:
reg_line = px.line(x=diabetes_X, y=model(diabetes_X))

plot = px.scatter(x=diabetes_X, y=y, labels={'x': 'BMI','y' : 'target'})
reg_line.update_traces(line=dict(color='red'))
plot.add_trace(reg_line.data[0])
plot.show()

In [31]:
# la regression est montée le long du y

Wesee that the values of the two parameters have changed, let's see how it affected the predictions of the model.
Visualize the data vs the model's predictions.

We notice the predictions got a little closer to our real data
* Recalculate your MSE

In [30]:
mse(model(diabetes_X), y)

# old value was 29225.594182761437

20626.788769167244

So the MSE dropped

* Our MSE has dropped a lot! This is good news but the process of gradient descent is iterative. So you'll have to do it several times before arriving at accurate predictions.
  * By making a loop, try to repeat the process from above 10,000 times.
  * Display every 1000 epochs: mse, model.beta_1 & model.beta_0

In [31]:
for n in range(10000):
  model.beta_1 -= learning_rate * derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)
  model.beta_0 -= learning_rate * derivative_mse_beta_0(model(diabetes_X), y)
  if n % 1000 == 0:
    print('current epoch is {}'.format(n))
    print('current beta_1 is : {}'.format(model.beta_1))
    print('current beta_0 is : {}'.format(model.beta_0))
    print('\n')


current epoch is 0
current beta_1 is : [0.84738912]
current beta_0 is : [55.14293086]


current epoch is 1000
current beta_1 is : [346.15411304]
current beta_0 is : [152.13348416]


current epoch is 2000
current beta_1 is : [565.76164008]
current beta_0 is : [152.13348416]


current epoch is 3000
current beta_1 is : [705.42722604]
current beta_0 is : [152.13348416]


current epoch is 4000
current beta_1 is : [794.25148524]
current beta_0 is : [152.13348416]


current epoch is 5000
current beta_1 is : [850.74177214]
current beta_0 is : [152.13348416]


current epoch is 6000
current beta_1 is : [886.66835958]
current beta_0 is : [152.13348416]


current epoch is 7000
current beta_1 is : [909.51688373]
current beta_0 is : [152.13348416]


current epoch is 8000
current beta_1 is : [924.04804556]
current beta_0 is : [152.13348416]


current epoch is 9000
current beta_1 is : [933.28954679]
current beta_0 is : [152.13348416]




* Using `plotly`, view your model and actual values again

In [32]:
reg_line = px.line(x=diabetes_X, y=model(diabetes_X))

plot = px.scatter(x=diabetes_X, y=y, labels={'x': 'BMI','y' : 'target'})
reg_line.update_traces(line=dict(color='red'))
plot.add_trace(reg_line.data[0])
plot.show()

**We've got a nice regression this time!**